In [1]:
import textract as tx
import nltk
from nltk.tokenize import word_tokenize #to clean and convert phrases into keywords
from nltk.corpus import stopwords

In [2]:
path = '/Users/nicolasdemuynck/BeCode/GNT-ARAI-1.25/Content/00.Use cases/ML6/data/10N_Sodium_Hydroxide_NaOH_40_6_US_EN_sds (1).pdf'
text = tx.process(path)
text = text.decode('utf-8')
print(text)

44.2.1

Page 1/11

Safety Data Sheet (SDS)
OSHA HazCom Standard 29 CFR 1910.1200(g) and GHS Rev 03.

Issue date 02/09/2017
*

Reviewed on 02/09/2017

1 Identification
· Product Identifier
· Trade name: 10N Sodium Hydroxide (NaOH 40%)
· Product Number: NGT-10N NaOH
· Relevant identified uses of the substance or mixture and uses advised against:
No further relevant information available.
· Product Description PC21 Laboratory chemicals
· Application of the substance / the mixture: Laboratory chemicals
· Details of the Supplier of the Safety Data Sheet:
· Manufacturer/Supplier:
NuGeneration Technologies, LLC (dba NuGenTec)
1155 Park Avenue, Emeryville, CA 94608
salesteam@nugentec.com
www.nugentec.com
1-888-996-8436 or 1-707-820-4080 for product information
· Emergency telephone number:
PERS Emergency Response: Domestic and Canada - 1-800-633-8253, International 1-801-629-0667

*

2 Hazard(s) Identification
· Classification of the substance or mixture:


~
d

GHS05 Corrosion

Skin Corr. 1A

### Break the string into string chunks

In [3]:
import re

# text = repr(text)
splitted_text = text.split('\n')
print(splitted_text)

['44.2.1', '', 'Page 1/11', '', 'Safety Data Sheet (SDS)', 'OSHA HazCom Standard 29 CFR 1910.1200(g) and GHS Rev 03.', '', 'Issue date 02/09/2017', '*', '', 'Reviewed on 02/09/2017', '', '1 Identification', '· Product Identifier', '· Trade name: 10N Sodium Hydroxide (NaOH 40%)', '· Product Number: NGT-10N NaOH', '· Relevant identified uses of the substance or mixture and uses advised against:', 'No further relevant information available.', '· Product Description PC21 Laboratory chemicals', '· Application of the substance / the mixture: Laboratory chemicals', '· Details of the Supplier of the Safety Data Sheet:', '· Manufacturer/Supplier:', 'NuGeneration Technologies, LLC (dba NuGenTec)', '1155 Park Avenue, Emeryville, CA 94608', 'salesteam@nugentec.com', 'www.nugentec.com', '1-888-996-8436 or 1-707-820-4080 for product information', '· Emergency telephone number:', 'PERS Emergency Response: Domestic and Canada - 1-800-633-8253, International 1-801-629-0667', '', '*', '', '2 Hazard(s) I

In [4]:
splitted_text[4]

'Safety Data Sheet (SDS)'

In [5]:
print(type(text))
print(type(splitted_text))

<class 'str'>
<class 'list'>


### Make dataframe with one column and each string sentence in a row 

In [29]:
import pandas as pd
import numpy as np
import textract
import re

while '' in splitted_text:
    splitted_text.remove('')

index = np.arange(0,len(splitted_text))
df_text = pd.DataFrame(splitted_text, index=[index], columns=['sentence'])
df_text.head(15)

,sentence
0,44.2.1
1,Page 1/11
2,Safety Data Sheet (SDS)
3,OSHA HazCom Standard 29 CFR 1910.1200(g) and G...
4,Issue date 02/09/2017
5,*
6,Reviewed on 02/09/2017
7,1 Identification
8,· Product Identifier
9,· Trade name: 10N Sodium Hydroxide (NaOH 40%)


In [147]:
def check_if_title(sentence):
    sentence = str(sentence)
    word_list = ['identification', 'information', 'measures', 'handling', 'properties', 'stability', 'considerations', 'exposure']
    
    starts_with_section = False
    starts_with_number = False
    if len(sentence.split()) > 1:
        starts_with_section = sentence.split()[0] == 'section'
        starts_with_number = sentence[0].isdigit() and not any(char.isdigit() for char in sentence[2:])
    
    
    contains_title_word = [ele for ele in word_list if(ele in sentence)] 
    
    if (starts_with_number or starts_with_section) and contains_title_word: 
        return 1
    else: 
        return 0

In [148]:
df_text["sentence"] = df_text["sentence"].str.lower()
df_text['title'] = df_text['sentence'].map(lambda a: check_if_title(a))

In [149]:
def dict_titles_with_values(data):
    lst = {}
    current_title = ''
    for i,j in zip(data['title'], data['sentence']):
        if i == 1:
            current_title = j
            lst[j] = []
        else:
            if current_title != '':
                lst[current_title].append(j)
    return lst

In [150]:
df1 = dict_titles_with_values(df_text)
df1

{'1 identification': ['· product identifier',
  '· trade name: 10n sodium hydroxide (naoh 40%)',
  '· product number: ngt-10n naoh',
  '· relevant identified uses of the substance or mixture and uses advised against:',
  'no further relevant information available.',
  '· product description pc21 laboratory chemicals',
  '· application of the substance / the mixture: laboratory chemicals',
  '· details of the supplier of the safety data sheet:',
  '· manufacturer/supplier:',
  'nugeneration technologies, llc (dba nugentec)',
  '1155 park avenue, emeryville, ca 94608',
  'salesteam@nugentec.com',
  'www.nugentec.com',
  '1-888-996-8436 or 1-707-820-4080 for product information',
  '· emergency telephone number:',
  'pers emergency response: domestic and canada - 1-800-633-8253, international 1-801-629-0667',
  '*'],
 '2 hazard(s) identification': ['· classification of the substance or mixture:',
  '\uf082',
  '~',
  'd',
  'ghs05 corrosion',
  'skin corr. 1a h314 causes severe skin burns

In [151]:
def listToString(s):
    str1 = " " 
    # return string   
    return (str1.join(s))

In [152]:
d2 = {}
for k, v in df1.items():
    d2[k] = listToString(v)

In [153]:
d2

{'1 identification': '· product identifier · trade name: 10n sodium hydroxide (naoh 40%) · product number: ngt-10n naoh · relevant identified uses of the substance or mixture and uses advised against: no further relevant information available. · product description pc21 laboratory chemicals · application of the substance / the mixture: laboratory chemicals · details of the supplier of the safety data sheet: · manufacturer/supplier: nugeneration technologies, llc (dba nugentec) 1155 park avenue, emeryville, ca 94608 salesteam@nugentec.com www.nugentec.com 1-888-996-8436 or 1-707-820-4080 for product information · emergency telephone number: pers emergency response: domestic and canada - 1-800-633-8253, international 1-801-629-0667 *',
 '2 hazard(s) identification': '· classification of the substance or mixture: \uf082 ~ d ghs05 corrosion skin corr. 1a h314 causes severe skin burns and eye damage. eye dam. 1 h318 causes serious eye damage. · label elements: · ghs label elements the produ